### Q.1) Python script for database connectivity. CSV loading, Perform CRUD operations. CSV should be in (utf-8) database in (latin-9). Read config file using argsparse then pass that to required functions. CSV should be more than 100000 rows

In [ ]:
import configparser
import pandas as pd
import pyodbc
import argparse
import time

config = configparser.ConfigParser()
config.read('config.ini')

def create_connection(server, database, username, password):
    conn = None
    try:
        conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server +
                              ';DATABASE=' + database +
                              ';UID=' + username +
                              ';PWD=' + password)
        print("Database connection established.")
    except pyodbc.Error as e:
        print(e)
    return conn

def create_table_from_dataframe(conn, df):
    try:
        cursor = conn.cursor()
        cursor.execute("IF OBJECT_ID('words', 'U') IS NOT NULL DROP TABLE words")
        column_types = {col: 'NVARCHAR(MAX)' for col in df.columns}
        create_table_sql = f"CREATE TABLE words ({', '.join([f'{col} {col_type}' for col, col_type in column_types.items()])})"
        cursor.execute(create_table_sql)
        conn.commit()
        print("Table created successfully based on DataFrame schema.")
    except pyodbc.Error as e:
        print(e)

def load_data(conn, df, batch_size=1000):
    try:
        cursor = conn.cursor()
        total_rows = len(df)
        start_time = time.time()
        for i in range(0, total_rows, batch_size):
            batch_df = df.iloc[i:i+batch_size]
            for _, row in batch_df.iterrows():
                row_values = [str(value) for value in row]
                cursor.execute("INSERT INTO words VALUES ({})".format(','.join('?' * len(row_values))), tuple(row_values))
            conn.commit()
            print(f"Loaded {min(i + batch_size, total_rows)} out of {total_rows} rows")
        end_time = time.time()
        print(f"Data loaded from DataFrame to database successfully. Time taken: {end_time - start_time:.2f} seconds")
    except pyodbc.Error as e:
        print(e)

def read_data(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM words")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except pyodbc.Error as e:
        print(e)

def delete_data(conn, Number):
    try:
        cursor = conn.cursor()
        cursor.execute("DELETE FROM words WHERE Number=?", (Number,))
        conn.commit()
        print("Data deleted successfully.")
    except pyodbc.Error as e:
        print(e)

def update_data(conn, Word, column, new_value):
    try:
        cursor = conn.cursor()
        update_query = f"UPDATE words SET {column} = ? WHERE Word = ?"
        cursor.execute(update_query, (new_value, Word))
        conn.commit()
        print("Data updated successfully.")
    except pyodbc.Error as e:
        print(e)

def main():
    parser = argparse.ArgumentParser(description="Database CRUD operations with words data")
    parser.add_argument("--config-file", type=str, help="Config file containing database connection information")
    parser.add_argument("--csv-file", type=str, help="CSV file containing words data")
    parser.add_argument("--read", action="store_true", help="Read data from the database")
    parser.add_argument("--update", nargs=3, metavar=('id', 'column', 'new_value'), help="Update data in the database")
    parser.add_argument("--delete", type=int, help="Delete data from the database by id")
    args = parser.parse_args()

    df = pd.read_csv(args.csv_file)
    username = config.get('Settings','username')
    password = config.get('Settings','password')
    driver_name = config.get('Settings','DRIVER_NAME')
    database_name = config.get('Settings','DATABASE_NAME')
    server_name = config.get('Settings','SERVER_NAME')
    conn = create_connection(server_name, database_name, username, password)
    if conn is not None:
        # create_table_from_dataframe(conn, df)  
        # load_data(conn, df)  

        if args.read:
            read_data(conn)
 
        if args.update:
            update_data(conn, *args.update)
 
        # if args.delete:
        #     delete_data(conn, args.delete)

        conn.close()
        print("Database connection closed.")

if __name__ == '__main__':
    main()

### Q2)Python code execution flow explanation

The flow of execution basically refers to the order in which statements are executed. That is to 
say, execution always starts at the first statement of the program. Moreover, statements execute 
one at a time. It happens in order, from top to bottom.
Further, functions definitions do not alter the flow of execution of the program. However, it 
remembers the statements inside the function do not execute until the functions is called.
Moreover, function calls are similar to a bypass in the flow of execution. Thus, instead of going to 
the next statement, the flow will jump to the first line of the called function. Then, it will execute all 
the statements there. After that, it will come back to pick up where it left off.

### Q3)List partition question.

In [4]:
l1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
partition_size = 4
result = [l1[i:i + partition_size] + [None] * (partition_size - len(l1[i:i + partition_size])) for i in range(0, 
len(l1), partition_size)]
print(result)

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, None, None]]


### Q4)Use Enum in python

In [6]:
from enum import Enum
class Role(Enum):
 MANAGER = "Manager"
 EMPLOYEE = "Employee"
 INTERN = "Intern"
class Employee:
 def __init__(self, name, role):
   self.name = name
   self.role = role
# Example usage:
emp1 = Employee("Alice", Role.MANAGER)
emp2 = Employee("Bob", Role.EMPLOYEE)
emp3 = Employee("Charlie", Role.INTERN)
print(emp1.name, "is a", emp1.role.value)
print(emp2.name, "is an", emp2.role.value)
print(emp3.name, "is an", emp3.role.value)

Alice is a Manager
Bob is an Employee
Charlie is an Intern


### Q5) Prepare notes for itertools functions which are commonly used.

Itertools is a module in Python, it is used to iterate over data structures that can be stepped over 
using a for-loop. Such data structures are also known as iterables. This module works as a fast, 
memory-efficient tool that is used either by themselves or in combination to form iterator algebra.
Different types of iterators provided by this module are:
• Infinite iterators
• Combinatoric iterators
• Terminating iterators
Infinite iterators
Iterator in Python is any Python type that can be used with a ‘for in loop’. Python lists, tuples, 
dictionaries, and sets are all examples of inbuilt iterators. But it is not necessary that an iterator 
object has to exhaust, sometimes it can be infinite. Such types of iterators are known as Infinite 
iterators.
count()
cycle() 
repeat() 
Combinatoric iterators
The recursive generators that are used to simplify combinatorial constructs such as permutations, 
combinations, and Cartesian products are called combinatoric iterators.
product()
permutations()
combinations()
combinations with replacements()
Terminating iterators
Terminating iterators are used to work on the short input sequences and produce the output based 
on the functionality of the method used.

### Q 6)Example of pass by reference and pass by value.

Pass by value: When passing immutable objects like integers, floats, strings, and tuples, a copy of 
the object is passed to the function. Any modifications made to the object within the function do 
not affect the original object.

In [8]:
def modify_value(x):
 x += 1
 print("Inside the function:", x)
value = 10
modify_value(value)
print("Outside the function:", value)

Inside the function: 11
Outside the function: 10


Pass by reference: When passing mutable objects like lists, dictionaries, and sets, the reference 
to the original object is passed. Modifications made to the object within the function affect the 
original object.

In [12]:
def modify_list(lst):
    lst.append(4)
    print("Inside the function:", lst)
my_list = [1, 2, 3]
modify_list(my_list)
print("Outside the function:", my_list)

Inside the function: [1, 2, 3, 4]
Outside the function: [1, 2, 3, 4]


### Q7)Demo of Union operator using typing library

In [13]:
from typing import Union
def square_or_string(x: Union[int, float, str]) -> Union[int, str]:
   if isinstance(x, (int, float)):
     return x * x
   elif isinstance(x, str):
     return x.upper()

print(square_or_string(5)) 
print(square_or_string(2.5)) 
print(square_or_string("hello"))

25
6.25
HELLO


### Q8)How to check if variable is iterable python.
To check if a variable is iterable in Python, use the iter() function along with a try-except block.

In [14]:
def is_iterable(obj):
   try:
     iter(obj)
     return True
   except TypeError:
     return False

print(is_iterable([1, 2, 3])) 
print(is_iterable("hello"))
print(is_iterable(5)) 

True
True
False


### Q9)What are various ways to implement string formatting in python.

### 1) Using the % Operator (Old Style Formatting):

In [15]:
name = "Alice"
age = 30
formatted_string = "Name: %s, Age: %d" % (name, age)
print(formatted_string)

Name: Alice, Age: 30


### 2) Using the str.format() Method (str.format() Method):

In [17]:
name = "Alice"
age = 30
formatted_string = "Name: {}, Age: {}".format(name, age)
print(formatted_string)

Name: Alice, Age: 30


### 3) Using f-strings (Formatted String Literals):

In [18]:
name = "Alice"
age = 30
formatted_string = f"Name: {name}, Age: {age}"
print(formatted_string)

Name: Alice, Age: 30


### 4) Using Template Strings (string.Template):

In [ ]:
name = "Alice"
age = 30
template = Template("Name: $name, Age: $age")
formatted_string = template.substitute(name=name, age=age)

### Q.10)Example of any() and all().

Any()-The any() function returns True if at least one element in the iterable is True, and False
otherwise.

In [20]:
# Example 1
numbers = [1, 2, 3, 4, 0]
result = any(numbers)
print(result) 
# Example 2
strings = ["hello", "", "world", ""]
result = any(strings)
print(result) 

True
True


All()-The all() function returns True if all elements in the iterable are True, and False otherwise

In [21]:
# Example 1
numbers = [1, 2, 3, 4, 0]
result = all(numbers)
print(result)
# Example 2
booleans = [True, True, True, True]
result = all(booleans)
print(result) 

False
True
